In [1]:
import pandas as pd




pd.set_option('display.max_rows', None)
# read exel file in desktop in my ubuntu and
df = pd.read_excel('./data/raw/Export Trade History.xlsx')
df['Date(UTC)'] = pd.to_datetime(df['Date(UTC)'])

####################### Extracting DATA ######################################
# Identify changes between zero and non-zero values
df['is_zero'] = df['Realized Profit'] == 0
df['group'] = df['is_zero'].ne(df['is_zero'].shift()).cumsum()

# Group by the identifier and sum 'New_Column_1' and 'Quantity'
result = df.groupby('group').agg({
    'Date(UTC)': 'first',
    'Symbol': 'first',
    'Side':'first',
    'Price': 'mean',
    'Quantity': 'sum',
    'Amount': 'sum',
    'Fee': 'sum',
    'Realized Profit': 'sum',
}).sort_values(by='group').reset_index(drop=True)

df_out = result[result['Realized Profit'] != 0].reset_index(drop=True)
df_out.columns = [col + '_OUT' if i < len(df_out.columns) - 1 else col for i, col in enumerate(df_out.columns)]
df_in = result[result['Realized Profit'] == 0].reset_index(drop=True)
df_in.columns = [col + '_IN' if i < len(df_in.columns) - 1 else col for i, col in enumerate(df_in.columns)]
df_in = df_in.drop('Realized Profit', axis=1)

################ Join the IN and OUT table ##############################
df_in = df_in.reset_index(drop=True)
df_out = df_out.reset_index(drop=True)
df_combined = pd.concat([df_in, df_out], axis=1)

############################## Feature Extraction #################################333
# Create 'Trade_Duration' column
df_combined['Trade_Duration'] = df_combined['Date(UTC)_OUT'] - df_combined['Date(UTC)_IN']
# Create 'Position' column
df_combined['Position'] = df_combined['Side_IN'].apply(lambda x: 'LONG' if x == 'BUY' else 'SHORT')
# Create 'Total_Fee' column
df_combined['Total_Fee'] = df_combined['Fee_IN'] + df_combined['Fee_OUT']
# Create 'Percentage' column
df_combined['Percentage'] = ((df_combined['Price_OUT'] - df_combined['Price_IN']) / df_combined['Price_IN']) * 100
# Create 'Earning' column
df_combined['Earning'] =   df_combined['Realized Profit'] - df_combined['Total_Fee']

df_combined.sort_values(by='Date(UTC)_IN',inplace=True)
df_combined.reset_index(drop=True, inplace=True)

# Create KASA
start_money = 927
df_combined['Total_Money'] = df_combined['Earning'].cumsum() + start_money

#dasdsa
df_combined['leverage'] = 100* df_combined['Realized Profit']/ df_combined.Percentage/20

# Final Representation
column_order = [
    'Date(UTC)_IN', 'Date(UTC)_OUT','Trade_Duration', 'Symbol_IN', 
    'Position','Price_IN', 'Price_OUT','Percentage', 'Quantity_IN', 
    'Amount_IN', 'Amount_OUT', 'Fee_IN', 'Fee_OUT','Total_Fee', 'Realized Profit','Earning','Total_Money', 'leverage'
]
df_combined = df_combined[column_order]

df_combined


/home/anil/.local/share/virtualenvs/trade_analysis-zrWQfRxp/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Date(UTC)_IN,Date(UTC)_OUT,Trade_Duration,Symbol_IN,Position,Price_IN,Price_OUT,Percentage,Quantity_IN,Amount_IN,Amount_OUT,Fee_IN,Fee_OUT,Total_Fee,Realized Profit,Earning,Total_Money,leverage
0,2023-11-16 12:28:10,2023-11-16 13:16:17,0 days 00:48:07,BTCUSDT,LONG,37113.400000,36961.700000,-0.408747,0.133,4936.08220,4915.90610,0.987216,0.000000,0.987216,-20.17610,-21.163316,905.836684,246.804110
1,2023-11-18 19:32:40,2023-11-18 20:14:03,0 days 00:41:23,LINKUSDT,SHORT,13.795000,13.668143,-0.919588,649.960,8966.19820,8883.62210,2.141991,4.441811,6.583802,82.57610,75.992298,981.828982,-448.984296
2,2023-11-19 10:12:23,2023-11-19 11:17:34,0 days 01:05:11,EDUUSDT,LONG,0.627800,0.623193,-0.733897,12742.000,7999.42760,7939.54180,1.629267,3.969771,5.599037,-59.88580,-65.484837,916.344144,407.998489
3,2023-11-19 11:22:59,2023-11-19 15:01:41,0 days 03:38:42,EDUUSDT,LONG,0.621565,0.629475,1.272532,8829.000,5487.95240,5557.42800,2.743976,2.778714,5.522690,69.47560,63.952910,980.297054,272.981845
4,2023-11-19 15:01:42,2023-11-19 15:03:10,0 days 00:01:28,EDUUSDT,SHORT,0.629022,0.628561,-0.073344,8829.000,5553.57670,5549.42670,2.776788,2.774713,5.551502,4.15000,-1.401502,978.895552,-282.911793
5,2023-11-19 15:03:10,2023-11-19 15:06:08,0 days 00:02:58,EDUUSDT,LONG,0.628716,0.628350,-0.058180,8829.000,5551.47970,5547.98170,2.775740,2.773991,5.549731,-3.49800,-9.047731,969.847822,300.616595
6,2023-11-19 15:14:28,2023-11-19 16:31:33,0 days 01:17:05,EDUUSDT,LONG,0.627600,0.631604,0.637961,12365.000,7760.27400,7809.58490,1.552055,3.904792,5.456847,49.31090,43.854053,1013.701874,386.472402
7,2023-11-19 16:32:40,2023-11-19 17:31:46,0 days 00:59:06,EDUUSDT,LONG,0.631700,0.634147,0.387315,13648.000,8621.44160,8653.36410,1.724288,4.326682,6.050970,31.92250,25.871530,1039.573404,412.100339
8,2023-11-19 17:38:54,2023-11-19 17:47:38,0 days 00:08:44,EDUUSDT,SHORT,0.638100,0.633089,-0.785318,1381.000,881.21610,874.29940,0.176243,0.437150,0.613393,6.91670,6.303307,1045.876711,-44.037601
9,2023-11-19 19:11:49,2023-11-19 20:55:28,0 days 01:43:39,EDUUSDT,LONG,0.628800,0.635644,1.088493,14546.000,9146.52480,9246.52720,1.829305,4.623264,6.452569,100.00240,93.549831,1139.426543,459.361674


In [8]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Date(UTC)_IN     31 non-null     datetime64[ns] 
 1   Date(UTC)_OUT    31 non-null     datetime64[ns] 
 2   Trade_Duration   31 non-null     timedelta64[ns]
 3   Symbol_IN        31 non-null     object         
 4   Position         31 non-null     object         
 5   Price_IN         31 non-null     float64        
 6   Price_OUT        31 non-null     float64        
 7   Percentage       31 non-null     float64        
 8   Quantity_IN      31 non-null     float64        
 9   Amount_IN        31 non-null     float64        
 10  Amount_OUT       31 non-null     float64        
 11  Fee_IN           31 non-null     float64        
 12  Fee_OUT          31 non-null     float64        
 13  Total_Fee        31 non-null     float64        
 14  Realized Profit  31 non-null

In [ ]:
import psycopg2
from psycopg2 import extras
import pandas as pd

# Database connection parameters
dbname = 'tradingDB'
user = 'anilberkdelikaya'
password = '277227'
host = 'localhost'
port = '5432'

# Create a connection
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Create a cursor object
cur = conn.cursor()

# Assuming df_sorted is your DataFrame
# Convert DataFrame to a list of tuples
tuples = [tuple(x) for x in df_sorted.to_numpy()]

# Define the insert query
# Make sure column names match your database schema
query = """
INSERT INTO your_table_name (column1, column2, column3, ...)
VALUES %s
"""

# Use execute_batch to insert the data
extras.execute_batch(cur, query, tuples)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

In [6]:
timestamp =34
output_folder = './data/processed/'
f"{output_folder}/Processed_Trade_History_{timestamp}.csv"

'./data/processed//Processed_Trade_History_34.csv'

,Date(UTC)_IN,Date(UTC)_OUT,Trade_Duration,Symbol_IN,Position,Price_IN,Price_OUT,Percentage,Quantity_IN,Amount_IN,Amount_OUT,Fee_IN,Fee_OUT,Total_Fee,Realized Profit,Earning,Total_Money,leverage
0,2023-11-16 12:28:10,2023-11-16 13:16:17,0 days 00:48:07,BTCUSDT,LONG,37113.400000,36961.700000,-0.408747,0.133,4936.08220,4915.90610,0.987216,0.000000,0.987216,-20.17610,-21.163316,905.836684,246.804110
1,2023-11-18 19:32:40,2023-11-18 20:14:03,0 days 00:41:23,LINKUSDT,SHORT,13.795000,13.668143,-0.919588,649.960,8966.19820,8883.62210,2.141991,4.441811,6.583802,82.57610,75.992298,981.828982,-448.984296
2,2023-11-19 10:12:23,2023-11-19 11:17:34,0 days 01:05:11,EDUUSDT,LONG,0.627800,0.623193,-0.733897,12742.000,7999.42760,7939.54180,1.629267,3.969771,5.599037,-59.88580,-65.484837,916.344144,407.998489
3,2023-11-19 11:22:59,2023-11-19 15:01:41,0 days 03:38:42,EDUUSDT,LONG,0.621565,0.629475,1.272532,8829.000,5487.95240,5557.42800,2.743976,2.778714,5.522690,69.47560,63.952910,980.297054,272.981845
4,2023-11-19 15:01:42,2023-11-19 15:03:10,0 days 00:01:28,EDUUSDT,SHORT,0.629022,0.628561,-0.073344,8829.000,5553.57670,5549.42670,2.776788,2.774713,5.551502,4.15000,-1.401502,978.895552,-282.911793
5,2023-11-19 15:03:10,2023-11-19 15:06:08,0 days 00:02:58,EDUUSDT,LONG,0.628716,0.628350,-0.058180,8829.000,5551.47970,5547.98170,2.775740,2.773991,5.549731,-3.49800,-9.047731,969.847822,300.616595
6,2023-11-19 15:14:28,2023-11-19 16:31:33,0 days 01:17:05,EDUUSDT,LONG,0.627600,0.631604,0.637961,12365.000,7760.27400,7809.58490,1.552055,3.904792,5.456847,49.31090,43.854053,1013.701874,386.472402
7,2023-11-19 16:32:40,2023-11-19 17:31:46,0 days 00:59:06,EDUUSDT,LONG,0.631700,0.634147,0.387315,13648.000,8621.44160,8653.36410,1.724288,4.326682,6.050970,31.92250,25.871530,1039.573404,412.100339
8,2023-11-19 17:38:54,2023-11-19 17:47:38,0 days 00:08:44,EDUUSDT,SHORT,0.638100,0.633089,-0.785318,1381.000,881.21610,874.29940,0.176243,0.437150,0.613393,6.91670,6.303307,1045.876711,-44.037601
9,2023-11-19 19:11:49,2023-11-19 20:55:28,0 days 01:43:39,EDUUSDT,LONG,0.628800,0.635644,1.088493,14546.000,9146.52480,9246.52720,1.829305,4.623264,6.452569,100.00240,93.549831,1139.426543,459.361674


In [13]:
df1=pd.DataFrame({'A':[1,2,3,3,1],'B':[2,3,4,4,2]})
df2=pd.DataFrame({'A':[1],'B':[2]})
df1

,A,B
0,1,2
1,2,3
2,3,4
3,3,4
4,1,2


In [14]:
df2

,A,B
0,1,2


In [19]:
df2[~df2.apply(tuple,1).isin(df2.apply(tuple,1))]

,A,B


In [15]:
df1[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]

,A,B
1,2,3
2,3,4
3,3,4


In [17]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Date(UTC)_IN     31 non-null     datetime64[ns] 
 1   Date(UTC)_OUT    31 non-null     datetime64[ns] 
 2   Trade_Duration   31 non-null     timedelta64[ns]
 3   Symbol_IN        31 non-null     object         
 4   Position         31 non-null     object         
 5   Price_IN         31 non-null     float64        
 6   Price_OUT        31 non-null     float64        
 7   Percentage       31 non-null     float64        
 8   Quantity_IN      31 non-null     float64        
 9   Amount_IN        31 non-null     float64        
 10  Amount_OUT       31 non-null     float64        
 11  Fee_IN           31 non-null     float64        
 12  Fee_OUT          31 non-null     float64        
 13  Total_Fee        31 non-null     float64        
 14  Realized Profit  31 non-null